In [42]:
import numpy as np
import matplotlib as plt

### Units definition

In [43]:
units='US' #'SI' #'US'


# Definición de Parámetros

In [44]:
d=0.03196
E=.5
He=500.  # Altura en metros  sobre el nivel del mar (ejemplo) [m]
mu_f=1.9822E-05 #Viscosidad cinemática del aire [Kg/(m·s)] (Tfilm=Temperatura del aire en °C)
Vw=0.611#Velocidad del viento en m/s
Rac1=6.40E-05 #[Ohm/m] Resistencia del conductor a 20 °C
Rac2=6.4E-05# [Ohm/m] Resistencia del conductor a 75 °C
T1=20 #Temperatura a la que se mide Rac1 en °C
T2=75 #Temperatura a la que se mide Rac2 en °C
Tav=75 #Temperatura objetivo promedio del conductor en °C
Ta=37.9 #Temperatura del aire en °C
Tc=75 #Temperatura del conductor en °C
alfa=0.5 #Albedo del conductor (valor típico entre 0 y 1)
phi=np.radians(90) #Ángulo de inclinación del conductor en radianes (0 radianes = horizontal, π/2 radianes = vertical)
Qse=1043 #Irradiancia solar en W/m^2 (valor típico)


Temperatura promedio frontera conductor (Temperatura ambiente y superficial del conductor)

In [45]:
Tfilm=(Tc+Ta)/2
print('Temperatura del aire en °C:', Tfilm)

Temperatura del aire en °C: 56.45


Cálculo de la densidad del aire

In [46]:

pf=(1.293-1.525E-4*He+6.379e-9*He**2)/(1+0.00367*Tfilm) #Densidad del aire [kg/m^3] 
print('Densidad del aire [kg/m^3] :', pf)

Densidad del aire [kg/m^3] : 1.0092557271274214


Viscosidad Dinámica del aire

In [47]:
Hf=(1.458e-6*(Tfilm+273.15)**1.5)/(Tfilm+273.15+110.4) 
print('Viscosidad dinámica del aire [kg/(m·s)] :', Hf)

Viscosidad dinámica del aire [kg/(m·s)] : 1.9828307803711403e-05


Conductividad Térmica

In [48]:
kf=2.424e-2+7.477e-5*Tfilm-4.407e-9*Tfilm**2 
print('Conductividad térmica del aire [W/(m·K)] :', kf)

Conductividad térmica del aire [W/(m·K)] : 0.0284467231427825


Cálculo de la Resistencia del condcutor a la temperatura objetivo.  

In [49]:

Rac=Rac1+(Tav-T1)*(Rac2-Rac1)/(T2-T1) #Resistencia del conductor [Ohm] (Tfilm=Temperatura del aire en °C)
print('Resistencia a Taverage del conductor [Ohm] :', Rac)


Resistencia a Taverage del conductor [Ohm] : 6.4e-05


In [50]:
kangle=1.194-np.cos(phi)+0.194*np.cos(2*phi)+0.368*np.sin(2*phi)
print('Ángulo del viento:',kangle)

Ángulo del viento: 1.0


In [51]:
NRE=d*pf*Vw/mu_f  #número de Reynolds [adimensional] (d=diámetro del conductor en metros, pf=densidad del aire en kg/m^3, Vw=velocidad del viento en m/s, mu_f=viscosidad dinámica del aire en kg/(m·s))
display('Número de Reynolds:' ,NRE)

'Número de Reynolds:'

994.2640382819264

Cálculo de la tasa de transferencia de calor por convección

In [52]:
qcn=3.645*pf**0.5*d**0.75*(Tc-Ta)**1.25 #Tasa de transferencia de calor por convección natural [W/m^2] (D=diámetro del conductor en metros, Tc=Temperatura del conductor en °C, Ta=Temperatura del aire en °C)
print('Tasa de transferencia de calor por convección natural [W/m^2] :', qcn)
qc1=(1.01+1.35*(NRE)**0.52)*kf*kangle*(Tc-Ta) #Tasa de transferencia de calor por convección forzada [W/m^2] (Vw=velocidad del viento en m/s, mu_f=viscosidad dinámica del aire en kg/(m·s), D=diámetro del conductor en metros, Tc=Temperatura del conductor en °C, Ta=Temperatura del aire en °C)
print('Tasa de transferencia de calor por convección forzada [W/m^2] :', qc1)
qc2=(0.754*(NRE)**0.6)*kf*kangle*(Tc-Ta) #Tasa de transferencia de calor por convección forzada [W/m^2] (Vw=velocidad del viento en m/s, mu_f=viscosidad dinámica del aire en kg/(m·s), D=diámetro del conductor en metros, Tc=Temperatura del conductor en °C, Ta=Temperatura del aire en °C)
print('Tasa de transferencia de calor por convección forzada [W/m^2] :', qc2)

Tasa de transferencia de calor por convección natural [W/m^2] : 25.343734420809987
Tasa de transferencia de calor por convección forzada [W/m^2] : 52.64111573922279
Tasa de transferencia de calor por convección forzada [W/m^2] : 50.035534157095874


In [53]:
qc=max(qcn, qc1, qc2)  #Tasa de transferencia de calor por convección [W/m^2]
display('Tasa de transferencia de calor por convección')
display(['Convección total qc: ',(qc),' [W/m^2]'])


'Tasa de transferencia de calor por convección'

['Convección total qc: ', np.float64(52.64111573922279), ' [W/m^2]']

Tasa de transferencia de calor por radiación

In [54]:

qr=17.8*E*d*(((Tc+273.15)/100)**4-((Ta+273.15)/100)**4)  #Tasa de transferencia de calor por radiación [W/m^2] (E=emissividad del conductor, Tc=Temperatura del conductor en °C, Ta=Temperatura del aire en °C)
print('Tasa de transferencia de calor por radiación')
print(['Radiación qr: ',(qr),' [W/m^2]'])


Tasa de transferencia de calor por radiación
['Radiación qr: ', 15.162322539640769, ' [W/m^2]']


Ganancia de calor por radiación solar

In [ ]:
qs=alfa*Qse*d  #Tasa de transferencia de calor por radiación solar [W/m^2] (alfa=albedo del conductor, Qse=irradiancia solar en W/m^2, theta=ángulo entre la normal a la superficie y la dirección del sol en radianes)
print('Tasa de transferencia de calor por radiación solar')
print(['Radiación solar qs: ',(qs),' [W/m^2]'])

Tasa de transferencia de calor por radiación solar
['Radiación solar qs: ', 16.66714, ' [W/m^2]']


Cálculo de la corriente del conductor para las condiciones establecidas

In [56]:

Icond=np.sqrt((qc + qr - qs)/ Rac)  
print('(Corriente máxima del conductor) [A] :', Icond)

(Corriente máxima del conductor) [A] : 893.8706061881904
